# RHESSys Workflow Starting From World Template
<br />
RHESSysWorkflows provides a series of Python tools for performing [RHESSys](https://github.com/RHESSys/RHESSys) data preparation workflows. These tools build on the workflow system defined by [EcohydroLib](https://github.com/selimnairb/EcohydroLib) and [RHESSysWorkflows](https://github.com/selimnairb/RHESSysWorkflows).
<br />


#### This notebook creates a RHESSys model starting by generating a world template file. The data preparation steps use an existing HydroShare resource. 

The general steps are:<br />
* Generate template <br />
* Create world <br />
* Create flow table <br />
* Initializing vegetation carbon and nitrogen stores <br />
* Creating a RHESSys TEC file <br />
* Running RHESSys models <br />


**Note: **Users interested in seeing step outputs, remove **output = ** from the command line. <br />
Some output lengths are very long and this notebook will become large in disk size.

This cell imports the necessary python libraries for the notebook 

In [ ]:
import os
import sys, stat
import logging
from utilities import hydroshare
import shutil
from utilities.rhessys import *
%matplotlib inline

print 'Finished Step'

The **`RHESSysWorkflow`** class is provided as part of the `rhessys` library (imported above) to streamline your interaction with the `RHESSysWorkflows` [core functions](https://github.com/selimnairb/RHESSysWorkflows).
 <br>

Create an instance of the `RHESSysWorkflow` class and assign it to the variable `w`. <br>
This command will create a directory for the given **project name** of `my_workflow` in the JupyterHub's default data directory. **If the directory exists, you will be prompted to use, or overwrite.** <br>
This notebook, a custom gage is used for **gageid**. <br>
A start date of **2008-01-01**, and an end date of **2010-01-01**.  

In [ ]:
w = RHESSysWorkflow(project_name='my_workflow', 
                    gageid='custom',
                    start_date='2008-01-01',
                    end_date='2008-01-07'
                    )

print 'Finished Step'

The RHESSysWorkflow class uses a logging library to document output and errors. We can display these messages in our notebook by attaching to the logger and redirecting the output to stdout. While this step is not necessary, it will provide us with more verbose output.

In [ ]:
# Get root logger for RHESSysWorkflow and set the output to stdout
root_logger = logging.getLogger()
root_logger.addHandler(logging.StreamHandler(sys.stdout))

print 'Finished Step'

## Retrieve data from HydroShare Resource

The first step to retrieve a resource from HydroShare is to create a connection as follows: <br>
**Note** you may be required to enter your credentials.

In [ ]:
hs = hydroshare.hydroshare()

print 'Finished Step'

The UNC Institute for the Environment has prepared a HydroShare Resource for [Dead Run](https://www.hydroshare.org/resource/6dbb0dfb8f3a498881e4de428cb1587c/) <br>
<br>
If you follow the web link, or move your mouse cursor over the above web link, you will note the web link has nas numbers and text after https://www.hydroshare.org/resource/. This text string is a resource-id. Here the resource id is 6dbb0dfb8f3a498881e4de428cb1587c and is needed to retrieve the resource from HydroShare. <br>
<br>
**Note:**  <br>
[1] If you use are using this notebook as a template for another **resource-id**, cells below, marked with &#9971; ,indicate where users will need to modify the cell to work with their data.  <br>
[2] If you have used this **resource-id** before in your userspace, you will be prompted to overwrite this data. <br>
 

In [ ]:
resource_id = '6dbb0dfb8f3a498881e4de428cb1587c'
content = hs.getResourceFromHydroShare(resource_id)

print 'Finished Step'



After the resource has finished downloading, the next step is to unzip the folder in your workspace. <br>
We know the name of the zipped file `RainCatchers_3m.zip` from the above step. <br>

&#9971; If you are using a different resource, you will be required to change the **hs.content['DR5_3m_nonburned_DEM_rain_duration_DEM_float_lctest_raingarden.zip']**, line below. <br>
Here, a new folder called `RHESSys_ETV` is created in the **workspace folder** created by the **RHESSysWorkflow object** to keep the data in a separate location from the following steps. The **zipname** variable, is the name defined in hs.content["MY_FILE"]. 

In [ ]:
#########################################################
## Variables used in cells below
zipfolder = w.sub_project_folder
zipname = 'DR5_3m_nonburned_DEM_rain_duration_DEM_float_lctest_raingarden'

## Need to update project location
w.sub_project_folder = zipfolder + '/' + zipname

#########################################################
## Commands
## Create Folder
w.create_path(zipfolder)
## name of zip file and path
zipfilepathname = hs.content[zipname + '.zip']
## Unzip the file
w.unzip_etv_zip_file_at_path(zipfilepathname, zipfolder)

print 'Finished Step'

The **os.environ["GDAL_DATA"]** indicates to PYTHON where the GDAL installation is. If you have compiled GDAL (i.e. for specialized libraries) in your environment, change this value.
<br/><br/>

In [ ]:
os.environ["GDAL_DATA"] = "/opt/conda/share/gdal"

print 'Finished Step'

## Download RHESSys Model software from HydroShare resource

The above resource contains a precompiled RHESSys model and tools. However, the code has been compiled on a OSX environment different OS from the Jupyter notebooks. Hence, the RHESSys model will not execute. <br />
<br />
As a notebook user, you have two options. <br />
 <br />
**Option #1:** As demonstrated with other RHESSys notebooks, a step called `ImportRHESSysSource` will import the latest RHESSys source code and compile the code on the host. This method is recommended if you desire the latest model and the data preparation steps are compatiable.
 <br />
**Option #2:** This notebook, demonstrates using an existing compiled RHESSys model and tools, kept on HydroShare as a resource called [rhessys5.18.r2](https://www.hydroshare.org/resource/f92ccd46170b4d4cac028b7765536d75/). <br>
<br>



In [ ]:
resource_id = 'f92ccd46170b4d4cac028b7765536d75' ##Resource id for rhessys5.18.r2
content = hs.getResourceFromHydroShare(resource_id)

print 'Finished Step'

In [ ]:
#########################################################
## Note dependency of variables [zipfolder] defined above 

zipname = 'rhessys5.18.r2'
bin_location = '/rhessys/bin'
zipfolder2 = w.sub_project_folder + bin_location

#########################################################
## Commands
## Create Folder
#w.create_path(zipfolder2)
## name of zip file and path
zipfilepathname = hs.content[zipname + '.zip']
## Unzip the file
w.unzip_etv_zip_file_at_path(zipfilepathname, zipfolder2)

print 'Finished Step'

The permissions of the file contents need to be modified for execution.

In [ ]:
#########################################################
## Note dependency of variables [zipfolder2] defined above 

#########################################################
## Commands
for filename in os.listdir(zipfolder2):
    file_bin = zipfolder2 + '/' + filename
    os.chmod(file_bin, 0777)

print 'Finished Step'

Here, the sub project folder is modified.

## Generate World Template

Now we are almost ready to create the world file for our watershed. First we must create the template from which the world file will be created. To do this, we'll use the **GenerateWorldTemplate** tool. Fortunately this is very easy because the metadata store contains nearly all the information needed to create the template.<br>
<br>
<br>
**Options** include the following: <br>
**-c (a.k.a. --climateStation)**      The climate station to associate with the worldfile. Must be one of the climate stations specified in the "climate_stations" key in the "rhessys" section of the metadata<br>
**--aspectMinSlopeOne**           Use slope map with a minimum value of 1.0 to be used for calculating spherical average aspect. Needed for areas of low slope due to limitations of RHESSys grass2world, which truncates slopes <1 to 0.0, which causes spherical average of <br><br>
**-v, --verbose **        Print detailed information about what the program is doing<br>

<br/> <br/>
&#9971; **Note: ** Modify **world_template_options** below to match your climate data inputs.


In [ ]:
#########################################################
## Variables
world_template_flags =  " -c dr5_composite5"
world_template_options = " -p " + w.sub_project_folder + world_template_flags

#########################################################
## Commands
output = w.GenerateWorldTemplateOptions(world_template_options)
print(output)

print 'Finished Step'

Now use the **CreateWorldfile** tool to create a world file using this template. <br/>
Note: Output from CreateWorldfile has been piped to output variable. For debugging, uncomment print statement (i.e. remove #).

In [ ]:
#########################################################
## Commands
output = w.CreateWorldfile(w.sub_project_folder)
#print(output) ##Note Output message Long

print 'Finished Step'

## Create Flow Table
As with worldfile creation, at this point in the workflow, RHESSysWorkflows's metadata store contains all the information needed to create a flow table using the createflowpaths (CF) RHESSys program. We'll use CreateFlowtable to create our flow table.
<br><br>
This will result in the creation of a flow table called world.flow in the flow directory of your rhessys directory. Now we have almost everything we need to run RHESSys simulations.
<br><br>
See the RHESSysWorkflows tutorial to learn how to [route surface flows for road pixels directly to the stream](https://github.com/selimnairb/RHESSysWorkflows#creating-the-flow-table) and [create surface flow tables using a roof connectivity map](https://github.com/selimnairb/RHESSysWorkflows#creating-a-surface-flow-table-using-a-roof-connectivity-map)
<br>
<br>
**Options** include the following: <br>
**--routeRoads**      Tell createflowpaths to route flow from roads to the nearest stream pixel (requires roads_rast to be defined in metadata)<br>
**--routeRoofs** Tell createflowpaths to route flow from roof tops based on roof top connectivity to nearest impervious surface
(requires roof_connectivity_rast and impervious_rast to be defined in metadata)<br>
**-f, --force ** Run createflowpaths even if DEM x resolution does not match y resolution<br>
**--ignoreBurnedDEM ** Ignore stream burned DEM, if present. Default DEM raster will be used for all operations. If not specified and if stream burned raster is present, stream burned DEM will be used for generating the flow table.<br>
**-v, --verbose **        Print detailed information about what the program is doing<br>


<br/> <br/>
&#9971; **Note: ** Modify **flow_table_options** below to match your data inputs.



In [ ]:
#########################################################
## Variables
flow_table_flags = " -v"
flow_table_options = " -p " + w.sub_project_folder + flow_table_flags

#########################################################
## Commands
output = w.CreateFlowtableOptions(flow_table_options)
print(output)

print 'Finished Step'

## Initializing vegetation carbon and nitrogen stores
RHESSys provides a program called LAIread to initialize vegetation carbon and nitrogen stores in your world file.

    Note, LAIread should only be used for RHESSys simulations with static vegetation (i.e. not dynamic vegetation mode enable via the -g command line option to RHESSys).

Initializing carbon and nitrogen stores is a multi-step process that involves running LAI read to generate a redefine worldfile, running a 3-day RHESSys simulation to incorporate the redefine worldfile, writing out a new worldfile with initialized vegetation carbon and nitrogen stores. RHESSysWorkflows automates all of these processes for you using the tool RunLAIRead, which can even figure out what date to start the 3-day RHESSys simulation on based on your climate data.

    In the current version of RHESSysWorkflows, RunLAIRead is only able to read simulation start dates from point time-series climate data. Users of ASCII or NetCDF gridded climate data must run LAI read by hand. The next release of RHESSysWorkflows will add support for gridded climate data to RunLAIRead.

To initialize vegetation carbon and nitrogen stores, LAIread relies on allometric relationships between leaf area and carbon and nitrogen mass in various plant tissues (e.g. leaf, stem, root). Consult the RHESSys wiki for more information on allometric relationships used by LAIread. These allometric parameters have not yet been added to the RHESSys ParamDB database proper. A default version of the parameters for RHESSys base vegetation classes is stored in the RHESSys ParamDB source coderepository. RHESSysWorkflows stores this file under the name allometric.txt in the allometry folder of the ParamDB of your rhessys/db folder. 


In [ ]:
#########################################################
## Commands
output = w.RunLAIRead(w.sub_project_folder)
#print(output) ##Note Output message VERY Long

print 'Finished Step'

## Creating a RHESSys TEC file
We need one more thing before we can run our model, a TEC file. TEC stands for "temporal event controller". We use a TEC file to tell RHESSys to do things on at certain times during a simulation.

#### With this notebook, the supplied **tec_daily.txt** will be used. If you desire a short test, or different times, uncomment the `output = w.RunCmdOptions(tec_options)` below.
<br/>
&#9971; **Note: ** Modify **options** below to match your data inputs.

In [ ]:
#########################################################
## Variables
model_start = str(w.model_start_year) + ' ' + str(w.model_start_month) + ' ' + str(w.model_start_day) + ' 1'

tec_file = "tec_daily.txt"
tec_output_location = w.sub_project_folder + "/rhessys/tecfiles/" + tec_file
tec_cmd = ' echo "' + model_start + ' print_daily_on" > '
tec_flags = tec_cmd + tec_output_location
tec_options = " -p " + w.sub_project_folder + tec_flags

#########################################################
## Commands
##output = w.RunCmdOptions(tec_options)
print(output)

print 'Finished Step'

## Running RHESSys models
Once you have built a RHESSys model using RHESSysWorkflows, you are free to run your model manually. However this will not capture information about model runs in your project metadata. If you would like to record your runs in your project metadata, use the **RunModel** command.


Because the project metadata knows where RHESSys is installed in your project directory, you don't have to specify the full path of any of the RHESSys input files (e.g. world files, flow tables, TEC files, etc), you only need to specify the filenames. RunModel will echo RHESSys console outlet to the screen (if the -v or verbose option is specified as above), and will always save the same output into a file named 'rhessys.out' stored in the output folder for each model run. The output folder will be named based on the value you provide for the '-pre' or output prefix option. Model output is stored in the output directory of the rhessys directory of your project directory.

For more details on using RunModel see the RHESSysWorkflows [tutorial](https://github.com/selimnairb/RHESSysWorkflows#running-rhessys-models).
<br>
<br>
**Options** include the following: <br>
**-d DESCRIPTION, --description** Description of the model run<br>
**-pre OUTPUTPREFIX** Filename prefix to use for output files, relative to output directory in the RHESSys directory of the project<br>
**-st STARTDATE STARTDATE STARTDATE STARTDATE** Start date and time of the model run, of the form "YYYY M D H"<br>
**-ed ENDDATE ENDDATE ENDDATE ENDDATE** Date date and time of the model run, of the form "YYYY M D H".<br>
**-w WORLDFILE** Filename of the tecfile to use for the model run, specified relative to the tec directory in the RHESSys directory of the projec<br>
**-t TECFILE **        Filename of the tecfile to use for the model run, specified relative to the tec directory in the RHESSys directory of the project<br>
**-r [FLOWTABLES [FLOWTABLES ...]]**Filename(s) of the flow table(s) to use for the model run, specified relative to the flowtable directory in the RHESSys directory of the project. If one flow table is supplied, it will be used for subsurface and surface routing. If two flow tables are supplied the first will be use for subsurface routing, the second for surface<br>
**--basin ** Tell RHESSys to output at the basin spatial level<br>
**--hillslope ** Tell RHESSys to output at the hillslope spatial level<br>
**--zone ** Tell RHESSys to output at the zone spatial level<br>
**--patch ** Tell RHESSys to output at the patch spatial level<br>
**--canopy ** Tell RHESSys to output at the canopy stratum spatial level<br>
**-v, --verbose **        Print detailed information about what the program is doing<br>


<br/> <br/>
&#9971; **Note: ** Modify **options** below to match your data inputs.


In [ ]:
#########################################################
## Variables
## Note dependency of variables [w.model_start_year, w.model_start_month, w.model_start_day] defined above 
## Note dependency of variables [w.model_end_year, w.model_end_month, w.model_end_day] defined above 

model_start = str(w.model_start_year) + ' ' + str(w.model_start_month) + ' ' + str(w.model_start_day) + ' 1'
model_end = str(w.model_end_year) + ' ' + str(w.model_end_month) + ' ' + str(w.model_end_day) + ' 1'

description = '"Test model run"'  ##Must use by quotes here
spatial_level = "--basin"
prefix = "test"
world_file = "world_init"
tec_file = "tec.lairead" 
flow_table = "world.flow"
s_flag = '0.07041256017 133.552915269 1.81282283058'
sv_flag = '4.12459677088 78.3440566535'
gw_flag = '0.00736592779294 0.340346799457'
flow_table_flag = ' -r ' + flow_table + ' -- -s ' + s_flag + ' -sv ' + sv_flag + ' -gw ' + gw_flag

runmodel_flags = '  -d ' + description + ' ' + spatial_level + ' -pre ' + prefix +' -st ' + model_start + ' -ed ' + model_end + ' -w ' + world_file + ' -t ' + tec_file + flow_table_flag
runmodel_options = " -v -p " + w.sub_project_folder + runmodel_flags
##print(runmodel_options)

#########################################################
## Commands
output = w.RunModelOptions(runmodel_options)
print(output)

print 'Finished Step'

## Help with error messages

If you see an error message, such as **returned non-zero exit status 1** check the log file located in the **w.sub_project_folder** folder. The log file name is the same name as the project folder, with **.log** appended.


## Resources

* RHESSys
  * [Setup](http://fiesta.bren.ucsb.edu/~rhessys/setup/setup.html)
  * [Wiki](http://fiesta.bren.ucsb.edu/~rhessys/)
* Data
  * [HydroShare](https://www.hydroshare.org/)
  * [USGS Data and Tools](https://www.usgs.gov/products/data-and-tools/data-and-tools-topics)
  * [USDA Data gateway](https://gdg.sc.egov.usda.gov/)
  * [HydroTerre](http://hydroterre.psu.edu/)

## Funding
This work was supported by the following NSF grants

    Award no. 1239678 EAGER: Collaborative Research: Interoperability Testbed-Assessing a Layered Architecture for Integration of Existing Capabilities

    Award no. 0940841 DataNet Federation Consortium.

    Award no. 1148090 Collaborative Research: SI2-SSI: An Interactive Software Infrastructure for Sustaining Collaborative Innovation in the Hydrologic Sciences


## Credits

**Lawrence E. Band** lband@email.unc.edu <br/>
**Brian Miles** brian_miles@unc.edu <br/> 
**Laurence Lin** laurence.lin@icloud.com <br/>
**John Duncan** jon.m.duncan@gmail.com <br/>
**Lorne Leonard** lnl3@psu.edu <br/>
